In [21]:
run create_map.py

Label(value='')

In [22]:
my_map

Map(center=[9.700554008484533, -0.541705192919114], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [19]:
def get_para_plot(para_name, x, y, xmin = 180, xmax = 330):
    x = np.array(x)
    y = np.array(y)
    
    para_min_maxs = {'DVS': [0, 2],
                     'LAI': [0, 3],
                     'TAGP': [0, 15000],
                     'TWSO': [0, 4000],
                     'TWLV': [0, 2000],
                     'TWST': [0, 10000],
                     'TWRT': [0, 2000],
                     'TRA':  [0, 0.5],
                     'RD':   [0, 100],
                     'SM':   [0, 0.8],
                     'WWLOW': [0, 100]
                    }

    ymin, ymax = para_min_maxs[para_name]
    
    mm = (x >= xmin) & (x <= xmax)
    x_scale = LinearScale(min = xmin, max = xmax)
    # ymin = np.maximum(np.nanpercentile(y[mm], 2.5) * 0.9, 0)
    # ymax = np.nanpercentile(y[mm], 97.5) * 1.1
    y_scale = LinearScale(min = ymin  , max = ymax)
    
    line = Lines(x=x, y=y, scales={"x": x_scale, "y": y_scale})
    tick_style = {'font-size': 8}
    tick_values = np.linspace(ymin, ymax, 4)
    tick_values
    
    ax_x = Axis(label="DOY", scale=x_scale,  num_ticks=5, tick_style=tick_style)
    ax_y = Axis(label=para_name, scale=y_scale, orientation="vertical", side="left", tick_values=tick_values, tick_style=tick_style)

    para_fig = Figure(layout=fig_layout, axes=[ax_x, ax_y], marks=[line], 
                       title=para_name, 
                       animation_duration=500, 
                       title_style = {'font-size': '8'},
                       fig_margin = dict(top=16, bottom=16, left=26, right=16))

    return para_fig


def update_wofost_fig_val(wofost_out_para, x, y, xmin = 180, xmax = 330):
    
    x = np.array(x)
    y = np.array(y)
    mm = (x >= xmin) & (x <= xmax)
    y = y[mm]
    x = x[mm]
    wofost_out_dict[wofost_out_para].marks[0].x = x
    wofost_out_dict[wofost_out_para].marks[0].y = y
    
wofost_out_paras = ['DVS', 'LAI', 'TAGP', 'TWSO', 'TWLV', 'TWST', 'TWRT', 'TRA', 'RD', 'SM', 'WWLOW']
para_figs = []

x = np.arange(180, 330)
y = np.zeros_like(x)

doys = [int(i.strftime('%j')) for i in df.index]
x = np.array(doys)
for wofost_out_para in wofost_out_paras:
    y = df.loc[:, wofost_out_para] * 0
    para_fig = get_para_plot(wofost_out_para, x, y)
    para_figs.append(para_fig)
wofost_out_dict = dict(zip(wofost_out_paras, para_figs))

wofost_out_panel_to_show = [wofost_out_dict['LAI'], wofost_out_dict['TWSO']]
wofost_out_panel = HBox(wofost_out_panel_to_show)
wofost_out_panel


In [20]:
update_wofost_fig_val('TWSO', doys, df.TWSO*1)
update_wofost_fig_val('LAI', doys, df.LAI*0.3)

In [9]:
from wofost_utils import create_ensemble, wofost_parameter_sweep_func, get_era5_gee

lon = -2.7
lat = 8.20
year = 2020

x_scale = LinearScale(min = 180, max = 365)
y_scale = LinearScale(min = 0  , max = 3)
x = np.arange(200, 365)
y = np.zeros_like(x)

line = Lines(x=x, y=y, scales={"x": x_scale, "y": y_scale})

tick_style = {'font-size': 8}
ax_x = Axis(label="DOY", scale=x_scale, num_ticks=5, tick_style=tick_style)
ax_y = Axis(label='LAI', scale=y_scale, orientation="vertical", side="left", num_ticks=4, tick_style=tick_style)

wofost_lai_fig = Figure(layout=fig_layout, axes=[ax_x, ax_y], marks=[line], 
                   title=field_id, 
                   animation_duration=500, 
                   title_style = {'font-size': '8'},
                   fig_margin = dict(top=16, bottom=16, left=16, right=16))
wofost_lai_fig



from map_utils import debounce

@debounce(0.2)
def on_change_wofost_slider(change):
    global lai_fig
    if (change['type'] == 'change'):
    
        meteo_file = get_era5_gee(year, lat, lon, dest_folder="data/")
        wofost_sliders_dict = dict(zip(paras, wofost_sliders))
        ens_parameters = {}
        paras_to_overwrite = [i for i in paras if 'AMAXTB_' not in i]
        for para in paras_to_overwrite:    
            ens_parameters[para] = wofost_sliders_dict[para].value
        ens_parameters['AMAXTB'] = [0, 55.0, 1.5, wofost_sliders_dict['AMAXTB_150'].value]
        df = wofost_parameter_sweep_func(year, ens_parameters = ens_parameters.copy(), meteo=meteo_file)
        dates = df.index
        doys = [int(i.strftime('%j')) for i in dates]
        
        # wofost_lai_fig.x = doys
        # wofost_lai_fig.y = np.array(df.LAI)
        wofost_lai_fig.marks[0].x = doys
        wofost_lai_fig.marks[0].y = np.array(df.LAI)
        
        # lai_fig.marks[1].scales = {'x': LinearScale(max=365.0, min=180.0), 'y': LinearScale(max=3.0, min=0.0)} 
        # lai_fig.axes[1].scale = LinearScale(max=3.0, min=0.0)
        # lai_fig.axes[0].scale = LinearScale(max=365.0, min=180.0)

        
for wofost_slider in wofost_sliders[:-1]:
    wofost_slider.observe(on_change_wofost_slider)
# plt.plot(dates, df.LAI)
# plt.twinx()
# plt.plot(dates, df.TWSO)

/home/users/marcyin/miniconda3/lib/python3.7/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [14]:
        meteo_file = get_era5_gee(year, lat, lon, dest_folder="data/")
        wofost_sliders_dict = dict(zip(paras, wofost_sliders))
        ens_parameters = {}
        paras_to_overwrite = [i for i in paras if 'AMAXTB_' not in i]
        for para in paras_to_overwrite:    
            ens_parameters[para] = wofost_sliders_dict[para].value
        ens_parameters['AMAXTB'] = [0, 55.0, 1.5, wofost_sliders_dict['AMAXTB_150'].value]
        df = wofost_parameter_sweep_func(year, ens_parameters = ens_parameters.copy(), meteo=meteo_file)
        dates = df.index
        doys = [int(i.strftime('%j')) for i in dates]
        
        # wofost_lai_fig.x = doys
        # wofost_lai_fig.y = np.array(df.LAI)
        wofost_lai_fig.marks[0].x = doys
        wofost_lai_fig.marks[0].y = np.array(df.LAI)

In [3]:
from ipywidgets import IntSlider
def read_wofost_data(fname):
    f = np.load(fname, allow_pickle=True)
    parameters = f.f.parameters
    t_axis = f.f.t_axis
    samples = f.f.samples
    lais = f.f.lais
    yields = f.f.yields
    DVS = f.f.DVS
    print("loading simulations")
    doys = [int(datetime.datetime.utcfromtimestamp(i.tolist()/1e9).strftime('%j')) for i in t_axis]
    return parameters, t_axis, samples, lais, yields, DVS, doys

parameters, t_axis, samples, lais, yields, DVS, simu_doys = read_wofost_data('data/wofost_sims_dvs150.npz')

k_slider1 = IntSlider(min=181, max=224, value=200,        # Opacity is valid in [0,1] range
               orientation='horizontal',       # Vertical slider is what we want
               readout=True,                # No need to show exact value
               layout=Layout(width='80%'),
               description='Doy of sowing: ', 
               style={'description_width': 'initial'}) 

k_slider2 = FloatSlider(min=0.05, max=0.55, value=0.35,       # Opacity is valid in [0,1] range
               step = 0.0025,
               orientation='horizontal',       # Vertical slider is what we want
               readout=True,                # No need to show exact value
               layout=Layout(width='80%'),
               description='Early stress level: ', 
               style={'description_width': 'initial'}) 

k_slider3 = FloatSlider(min=0.05, max=0.55,  value=0.35,       # Opacity is valid in [0,1] range
               step = 0.0025,
               orientation='horizontal',       # Vertical slider is what we want
               readout=True,                # No need to show exact value
               layout=Layout(width='80%'),
               description='Late stress level: ', 
               style={'description_width': 'initial'}) 

def on_change_k_slider(change):

    if (change['name'] == 'value') & (change['type'] == 'change'):
        value = change["new"]
        old = change['old']
        
        k1 = k_slider1.value
        k2 = k_slider2.value
        k3 = k_slider3.value
        
        
        diff = abs(parameters - np.array([[k1, k2, k3]])).sum(axis=1)
        ind = np.argmin(diff)
        
        simu_lai = lais[ind]
        
        
        # var_line = line_axs[-1]
        # var_line.scales = line_axs[5][0].scales
        # field_lai_boxes.scales = var_line.scales
        # lai_dot.scales = var_line.scales

        wofost_lai.x = simu_doys
        wofost_lai.y = simu_lai
        wofost_lai.scales = lai_fig.marks[1].scales
        wofost_lai.colors = ['red']
        
        print(simu_doys[np.nanargmin(abs(DVS[ind]-1.25))])
        
        # print(k1, k2, k3)
        # print(parameters[ind])
        
        
        
k_slider1.observe(on_change_k_slider)
k_slider2.observe(on_change_k_slider)
k_slider3.observe(on_change_k_slider)

panel_box = VBox([fig_box, k_slider1, k_slider2, k_slider3], layout = box_layout)
panel_box

loading simulations
